In [14]:
import numpy as np
import xarray as xr
import pandas as pd

In [24]:
nctest = netCDF4.Dataset('Datasets/gdhy_v1.2_v1.3_20190128/rice/yield_2016.nc4')

In [25]:
nctest.variables.keys()

dict_keys(['lon', 'lat', 'var'])

In [26]:
lon = nctest.variables['lon']
lat = nctest.variables['lat']
yields = nctest.variables['var']

In [3]:
import nctoolkit as nc
from netCDF4 import Dataset

In [4]:
file_name = 'Datasets/gdhy_v1.2_v1.3_20190128/rice/yield_2016.nc4'

In [5]:
file_id = Dataset(file_name)

In [6]:
#ds = nc.open_data('Datasets/gdhy_v1.2_v1.3_20190128/rice/yield_2016.nc4')
#ds.tmean()
#df = ds.to_dataframe()


In [10]:
lon = file_id.variables['lon'][:]
lat = file_id.variables['lat'][:]
var = file_id.variables['var'][:]

In [27]:
file_id.variables['var'].dimensions

('lat', 'lon')

In [19]:
print(np.min(lon))
print(np.max(lon))
print(lon.shape)

0.25
359.75
(720,)


In [20]:
print(np.min(lat))
print(np.max(lat))
print(lat.shape)


-89.75
89.75
(360,)


In [21]:
print(np.min(var))
print(np.max(var))
print(var.shape)

0.0
25.44142
(360, 720)


In [22]:
lons, lats = np.meshgrid(lon, lat)

In [28]:
lats.shape

(360, 720)

In [59]:
lon_all = lons.reshape(-1,)

In [60]:
lon_all.shape

(259200,)

In [51]:
lon_all = lons.reshape(-1,1)
lat_all = lats.reshape(-1,1)
yields = var.reshape(-1,1)

lon_all = np.squeeze(lon_all)
lat_all = np.squeeze(lat_all)
yields = np.squeeze(yields)

In [52]:
lon_all.shape

(259200,)

In [53]:
yields.shape

(259200,)

In [54]:
df = pd.DataFrame({'longitude': lon_all, 'latitude': lat_all, 'Yield': yields})

In [55]:
df.head()

,longitude,latitude,Yield
0,0.25,-89.75,NaN
1,0.75,-89.75,NaN
2,1.25,-89.75,NaN
3,1.75,-89.75,NaN
4,2.25,-89.75,NaN


In [56]:
df_filtered = df.dropna(subset=['Yield'])
df_filtered

,longitude,latitude,Yield
74014,287.25,-38.75,7.805690
74734,287.25,-38.25,8.472213
74735,287.75,-38.25,9.086303
75453,286.75,-37.75,12.492006
75454,287.25,-37.75,11.382429
...,...,...,...
201131,125.75,49.75,11.525588
201132,126.25,49.75,11.951449
201133,126.75,49.75,9.320266
201851,125.75,50.25,11.057899


In [57]:
from netCDF4 import Dataset
import os

In [62]:
directory = 'Datasets/gdhy_v1.2_v1.3_20190128'
folders = ['maize_major','maize_second','rice_major','rice_second','soybean_major','wheat_spring','wheat_winter']
file_n = 'yield_2016.nc4'

In [64]:
dataframes_list = []

In [65]:
for folder_n in folders:
    file_path = os.path.join(directory, folder_n, file_n)

    file_id = Dataset(file_path)

    lon = file_id.variables['lon'][:]
    lat = file_id.variables['lat'][:]
    var = file_id.variables['var'][:]

    lons, lats = np.meshgrid(lon, lat)

    lon_all = lons.reshape(-1,)
    lat_all = lats.reshape(-1,)
    yields = var.reshape(-1,)

    df = pd.DataFrame({'longitude': lon_all, 'latitude': lat_all, 'Yield': yields})
    
    dataframes_list.append(df)

    

In [71]:
seasons = ['major','second','spring','winter']

In [72]:
dataframes_list[0]['crop'] = 'maize'
dataframes_list[1]['crop'] = 'maize'
dataframes_list[2]['crop'] = 'rice'
dataframes_list[3]['crop'] = 'rice'
dataframes_list[4]['crop'] = 'soybean'
dataframes_list[5]['crop'] = 'wheat'
dataframes_list[6]['crop'] = 'wheat'

In [73]:
dataframes_list[0]['season'] = seasons[0]
dataframes_list[1]['season'] = seasons[1]
dataframes_list[2]['season'] = seasons[0]
dataframes_list[3]['season'] = seasons[1]
dataframes_list[4]['season'] = seasons[0]
dataframes_list[5]['season'] = seasons[2]
dataframes_list[6]['season'] = seasons[3]

In [80]:
dataframes_list[6].dropna(subset=['Yield'])

,longitude,latitude,Yield,crop,season
68255,287.75,-42.75,5.905015,wheat,winter
70413,286.75,-41.25,7.615662,wheat,winter
70415,287.75,-41.25,5.689150,wheat,winter
71133,286.75,-40.75,7.876403,wheat,winter
71134,287.25,-40.75,7.362704,wheat,winter
...,...,...,...,...,...
212431,15.75,57.75,8.024502,wheat,winter
212451,25.75,57.75,2.422267,wheat,winter
213150,15.25,58.25,7.755891,wheat,winter
213151,15.75,58.25,7.074670,wheat,winter


In [81]:
yield_df = pd.DataFrame()

for df in dataframes_list:
    yield_df = pd.concat([yield_df, df.dropna(subset=['Yield'])], axis=0)

In [82]:
yield_df

,longitude,latitude,Yield,crop,season
70413,286.75,-41.25,26.728754,maize,major
71133,286.75,-40.75,25.878883,maize,major
71134,287.25,-40.75,26.511133,maize,major
71153,296.75,-40.75,1.549129,maize,major
71853,286.75,-40.25,22.426491,maize,major
...,...,...,...,...,...
212431,15.75,57.75,8.024502,wheat,winter
212451,25.75,57.75,2.422267,wheat,winter
213150,15.25,58.25,7.755891,wheat,winter
213151,15.75,58.25,7.074670,wheat,winter
